# 서울 공동주택 실거래가 예측

## Contents
- Library Import
- Data Load
- Data Preprocessing
- Feature Engineering
- Model Training
- Inference
- Output File Save

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/font/NanumGothic.otf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from lightgbm import LGBMRegressor

import eli5
from eli5.sklearn import PermutationImportance

from datetime import datetime
import time
from zoneinfo import ZoneInfo
import os

## 2. Data Load

#### 2.1. 데이터 로드

In [ ]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = './data/train.csv'
test_path  = './data/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', train_df.shape, 'Test data shape : ', test_df.shape)

## 3. Data Preprocessing

- 모델링 전에 데이터 내 결측치, 이상치 등을 제거하고 범주형과 연속형 변수를 살펴보도록 하겠습니다!
- 먼저, 용이한 전처리를 위해 train과 test data를 합친 하나의 데이터로 진행하도록 하겠습니다.

In [ ]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train_df['is_test'] = 0
test_df['is_test'] = 1
concat = pd.concat([train_df, test_df])     # 하나의 데이터로 만들어줍니다.

In [ ]:
concat['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

In [ ]:
# EDA를 통해 사용 결정한 필드들만 뽑아서 새로운 DataFrame 생성
selected_fields = [
    '시군구', '번지', '본번', '부번', '도로명', '아파트명',
    '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
    'target', 
    'is_test'
    ]

concat_selected_df = concat[selected_fields].copy()

In [ ]:
concat_selected_df = concat_selected_df.rename(columns={'전용면적(㎡)':'전용면적'})

In [ ]:
concat_selected_df

In [ ]:
concat_selected_df.info()

In [ ]:
concat_selected_df['본번'] = concat_selected_df['본번'].astype('str')
concat_selected_df['부번'] = concat_selected_df['부번'].astype('str')

concat_selected_df['target'] = concat_selected_df['target'].fillna(0)
concat_selected_df['target'] = concat_selected_df['target'].astype('int64')

concat_selected_df.info()

### 지하철 데이터 병합

In [ ]:
train_subway_path = './data/train_with_subway_infos.csv'
train_subway_df = pd.read_csv(train_subway_path)
print(train_subway_df.shape)
print(train_subway_df.info())
print(train_subway_df.head(3))

In [ ]:
test_subway_path  = './data/test_with_subway_infos.csv'
test_subway_df = pd.read_csv(test_subway_path)
print(test_subway_df.shape)
print(test_subway_df.info())
print(test_subway_df.head(3))

In [ ]:
total_subway_df = pd.concat([train_subway_df, test_subway_df], axis=0) 
print(total_subway_df.shape)
print(total_subway_df.info())
print(total_subway_df.head(3))

In [ ]:
concat_selected_df = pd.concat([concat_selected_df, total_subway_df], axis=1) 
print(concat_selected_df.shape)
print(concat_selected_df.info())
print(concat_selected_df.head(3))

### 버스 데이터 병합

In [ ]:
train_bus_path = './data/train_with_bus_infos.csv'
train_bus_df = pd.read_csv(train_bus_path)
print(train_bus_df.shape)
print(train_bus_df.info())
print(train_bus_df.head(3))

In [ ]:
test_bus_path  = './data/test_with_bus_infos.csv'
test_bus_df = pd.read_csv(test_bus_path)
print(test_bus_df.shape)
print(test_bus_df.info())
print(test_bus_df.head(3))

In [ ]:
total_bus_df = pd.concat([train_bus_df, test_bus_df], axis=0) 
print(total_bus_df.shape)
print(total_bus_df.info())
print(total_bus_df.head(3))

In [ ]:
# 지하철 정보에서 이미 머지되었으므로 버스에서는 삭제
del total_bus_df['좌표X_2']
del total_bus_df['좌표Y_2']

In [ ]:
concat_selected_df = pd.concat([concat_selected_df, total_bus_df], axis=1) 
print(concat_selected_df.shape)
print(concat_selected_df.info())
print(concat_selected_df.head(3))

## 5. Model Training

- 이제 위에서 만든 파생변수들과 정제한 데이터를 기반으로 본격적으로 부동산 실거래가를 예측하는 모델링을 진행하겠습니다.
- 모델링에는 LGBM을 이용하도록 하겠습니다.

### 5.1. 변수 Encoding
- 범주형 변수는 그대로 모델에 투입하면, 모델이 제대로 작동할 수 없습니다.
- 따라서 **레이블 인코딩 과정**을 통해 범주형 변수들을 numeric하게 바꾸는 인코딩 과정을 진행해주도록 하겠습니다.

In [ ]:
continuous_features = []
categorical_features = []

for column in concat_selected_df.columns:
    if pd.api.types.is_numeric_dtype(concat_selected_df[column]):
        continuous_features.append(column)
    else:
        categorical_features.append(column)

continuous_features.remove('target')
continuous_features.remove('is_test')

print("연속형 변수:", continuous_features)
print("범주형 변수:", categorical_features)

In [ ]:
# 범주형 변수 인코딩

le = LabelEncoder()
for col in categorical_features:
    concat_selected_df[col] = le.fit_transform(concat_selected_df[col].astype(str))

In [ ]:
for col in categorical_features:
    print(f"{col}: {concat_selected_df[col].shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures

# 연속형 변수 인코딩

# 1. StandardScaler
scaler = StandardScaler()
concat_selected_df[continuous_features] = scaler.fit_transform(concat_selected_df[continuous_features])

# 2. MinMaxScaler (선택적)
# scaler = MinMaxScaler()
# df[continuous_features] = scaler.fit_transform(df[continuous_features])

# 3. RobustScaler (선택적)
# scaler = RobustScaler()
# df[continuous_features] = scaler.fit_transform(df[continuous_features])

# 4. 로그 변환 (양수 값에 대해서만 적용 가능)
# for feature in continuous_features:
#     if (concat_selected_df[feature] > 0).all():
#         concat_selected_df[f'{feature}_log'] = np.log1p(concat_selected_df[feature])

# # 5. 이산화 (예: '전용면적'을 구간으로 나누기)
# concat_selected_df['전용면적_구간'] = pd.cut(concat_selected_df['전용면적'], bins=5, labels=['매우작음', '작음', '중간', '큼', '매우큼'])

# # 6. 다항식 특성 (선택적)
# poly = PolynomialFeatures(degree=2, include_bias=False)
# poly_features = poly.fit_transform(concat_selected_df[continuous_features])
# poly_features_names = poly.get_feature_names_out(continuous_features)
# df_poly = pd.DataFrame(poly_features, columns=poly_features_names)
# df = pd.concat([concat_selected_df, df_poly], axis=1)

# 결과 확인
# print(df[continuous_features + [f'{feature}_log' for feature in continuous_features] + ['전용면적_구간']].head())
# print(df_poly.head())

In [ ]:
concat_selected_df.head(1)        # 인코딩이 된 모습입니다.

In [ ]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_selected_df.query('is_test==0')
dt_test = concat_selected_df.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

dt_test.head(1)

### 5.2. Model Training
- 위 데이터를 이용해 모델을 train 해보겠습니다. 모델은 LightGBM 이용하겠습니다.
- Train과 Valid dataset을 분할하는 과정에서는 `holdout` 방법을 사용하겠습니다. 이 방법의 경우  대략적인 성능을 빠르게 확인할 수 있다는 점에서 baseline에서 사용해보도록 하겠습니다.
  - 이 후 추가적인 eda를 통해서 평가세트와 경향을 맞추거나 kfold와 같은 분포에 대한 고려를 추가할 수 있습니다.

In [ ]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [ ]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [ ]:
train_time = datetime.fromtimestamp(time.time(), tz=ZoneInfo("Asia/Seoul")).strftime("%Y%m%d-%H%M%S")
train_time

In [ ]:
# LightGBM 모델 정의
model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=1,
    n_jobs=-1
)

# 모델 학습
model.fit(X_train, y_train)

- 랜덤포레스트의 하이퍼파라미터도 데이터에 맞게 지정해줄 수 있습니다. 데이터에 맞는 하이퍼파라미터를 찾는 것도 성능 향상에 도움이 될 수 있습니다.

In [ ]:
# 훈련 데이터에 대한 예측
y_train_pred = model.predict(X_train)

# 검증 데이터에 대한 예측
pred = model.predict(X_val)

- 변수 중요도도 확인해보도록 하겠습니다.

In [ ]:
# 특성 중요도 출력
importance = model.feature_importances_
feature_names = model.feature_name_
for name, importance in sorted(zip(feature_names, importance), key=lambda x: x[1], reverse=True):
    print(f'{name}: {importance}')

In [ ]:
model_file_path = os.path.join('model', f'{train_time}.pkl')

# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open(model_file_path, 'wb') as f:
    pickle.dump(model, f)

### 5.4. 평가 및 분석

- 훈련, 검증, 테스트 세트에 대해 모델을 평가합니다.

In [ ]:
import import_ipynb
import evaluator

# get [interpretation, rmse, r2, mae]
train_result = evaluator.evaluate_set(y_train, y_train_pred, "Train")
val_result = evaluator.evaluate_set(y_val, pred, "Valid")

comprehensive_report = evaluator.comprehensive_evaluation(train_result, val_result)
print(comprehensive_report)

In [ ]:
# Validation dataset에 target과 pred 값을 채워주도록 하겠습니다.
X_val['target'] = y_val
X_val['pred'] = pred

In [ ]:
# Squared_error를 계산하는 함수를 정의하겠습니다.
def calculate_se(target, pred):
    squared_errors = (target - pred) ** 2
    return squared_errors

# RMSE 계산
squared_errors = calculate_se(X_val['target'], X_val['pred'])
X_val['error'] = squared_errors

In [ ]:
# Error가 큰 순서대로 sorting 해 보겠습니다.
X_val_sort = X_val.sort_values(by='error', ascending=False)       # 내림차순 sorting

In [ ]:
X_val_sort.head()

## 6. Inference

In [ ]:
dt_test.head(2)      # test dataset에 대한 inference를 진행해보겠습니다.

In [ ]:
# 저장된 모델을 불러옵니다.
with open(model_file_path, 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

In [ ]:
real_test_pred          # 예측값들이 출력됨을 확인할 수 있습니다.

## 7. Output File Save

In [ ]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
submission_file_path = os.path.join('output', f'{train_time}.csv')
preds_df.to_csv(submission_file_path, index=False)